<a href="https://colab.research.google.com/github/hrbae/LG_AI_training/blob/main/M2_System%20Optimization%20by%20ML/LG_Day2_Reinforcement%20Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### 과제. 작업자 배정 문제 - 추가 제약사항 

- 목적: 총 비용 최소화
- 제약조건 1: 각 작업자를 최소 1개, 최대 3개 작업에 할당 (기존) -> 각 작업자를 최소 2개, 최대 3개 작업에 할당 (변경)
- 제약조건 2: 각 작업은 최소 1명의 작업자에게만 할당이 되어야 함   
- 제약조건 3: 작업자는 2개의 팀으로 분할되며 (기존) -> 작업자는 3개의 팀으로 분할되며 (변경), 각 팀은 최소 2개 이상 최대 4개까지 작업할당이 가능

| Worker | Team   | Task 0 | Task 1 | Task 2 | Task 3 | Task 4 | Task 5 | Task 6 | Task 7 | Task 8 | Task 9 | Task 10 |
| ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------ | ------  |
| 0      | Team 1 | 90     | 80     | 75     | 100    | 100    | 140    | 120    | 35     | 210    | 130    | 80      |
| 1      | Team 1 | 65     | 35     | 55     | 165    | 120    | 55     | 40     | 45     | 180    | 220    | 130     |
| 2      | Team 2 | 225    | 45     | 90     | 55     | 140    | 100    | 60     | 15     | 140    | 100    | 20      |
| 3      | Team 2 | 35     | 20     | 95     | 315    | 50     | 80     | 80     | 35     | 100    | 60     | 90      |
| 4      | Team 3 | 10     | 150    | 90     | 120    | 40     | 200    | 90     | 95     | 300    | 50     | 100     |
| 5      | Team 3 | 80     | 40     | 100    | 30     | 70     | 150    | 30     | 15     | 400    | 90     | 150     |

##### 기존 코드

In [5]:
from ortools.linear_solver import pywraplp
from ortools.constraint_solver import pywrapcp

In [7]:
costs = [
    [90, 80, 75, 100, 100, 140, 120, 35, 210, 130],
    [65, 35, 55, 165, 120, 55, 40, 45, 180, 220],
    [225, 45, 90, 55, 140, 100, 60, 15, 140, 100],
    [35, 20, 95, 315, 50, 80, 80, 35, 100, 60],
    [10, 150, 90, 120, 40, 200, 90, 95, 300, 50],
    [80, 40, 100, 30, 70, 150, 30, 15, 400, 90]
]

num_workers = """과제 조건 반영"""
num_tasks = """과제 조건 반영"""
min_task_for_worker = """과제 조건 반영"""
max_task_for_worker = """과제 조건 반영"""
team1 = """과제 조건 반영"""
team2 = """과제 조건 반영"""
team3 = """과제 조건 반영"""
team_min = """과제 조건 반영"""
team_max = """과제 조건 반영"""


solver = pywraplp.Solver.CreateSolver('SCIP')

x = {}
for worker in range(num_workers):
    for task in range(num_tasks):
        x[worker, task] = solver.BoolVar(f'x[{worker},{task}]')
        
# Constraint 1
for worker in range(num_workers):
    solver.Add(solver.Sum([x[worker, task] for task in range(num_tasks)]) >= min_task_for_worker)
    solver.Add(solver.Sum([x[worker, task] for task in range(num_tasks)]) <= max_task_for_worker)
    
    
# Constraint 2
for task in range(num_tasks):
    solver.Add(solver.Sum([x[worker, task] for worker in range(num_workers)]) == 1)
    
# Constraint 3
team1_tasks = []
for worker in team1:
    for task in range(num_tasks):
        team1_tasks.append(x[worker, task])
solver.Add(solver.Sum(team1_tasks) >= team_min)
solver.Add(solver.Sum(team1_tasks) <= team_max)


team2_tasks = []
for worker in team2:
    for task in range(num_tasks):
        team2_tasks.append(x[worker, task])
########################## 빈칸 ##########################
solver.Add(solver.Sum(team2_tasks) >= team_min)
solver.Add(solver.Sum(team2_tasks) <= team_max)
##########################################################

objective_terms = []
for worker in range(num_workers):
    for task in range(num_tasks):
        objective_terms.append(costs[worker][task] * x[worker, task])
solver.Minimize(solver.Sum(objective_terms))

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL or status == pywraplp.Solver.FEASIBLE:
    print(f'Total cost = {solver.Objective().Value()}\n')
    for worker in range(num_workers):
        for task in range(num_tasks):
            if x[worker, task].solution_value() > 0.5:
                print(f'Worker {worker} assigned to task {task}.' +
                      f' Cost = {costs[worker][task]}')
else:
    print('No solution found.')
print(f'Time = {solver.WallTime()} ms')

Total cost = 265.0

Worker 0 assigned to task 2. Cost = 75
Worker 1 assigned to task 5. Cost = 55
Worker 2 assigned to task 1. Cost = 45
Worker 3 assigned to task 4. Cost = 50
Worker 4 assigned to task 0. Cost = 10
Worker 5 assigned to task 3. Cost = 30
Time = 8 ms
